What is the most problematic site?
What is the most problematic node?
What are the top affected nodes in the site Z?
What are the recently correlated alarms?
What are the top reported alarms in the node A?
Have any anomalies been reported on the node B?
What are the top affected sites?
How to resolve the alarm BGP session down?
Why does the alarm BFD session down occur?
Why does the link down alarm occur?
Monitor the network using RSRP and SINR metrics and ensure network is stable
Raise a high priority ticket when alarm X and alarm Y are reported within a span of 5 minutes
Provide the details of alarms reported in site A over last 2 hours?
Get the list of devices with license expiry due in 1 month
Monitor bandwidth utilization metric in devices of type C, verify the configuration parameters of misbaving devices and update the configurations once found
What are the sites having devices that have the risk of capacity overruns

In [1]:
from langchain.prompts import ChatPromptTemplate,FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.output_parsers.list import ListOutputParser
from langchain.output_parsers import CommaSeparatedListOutputParser

In [2]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [3]:
examples = [
    {"input": "What is the most problematic site", "output": ["find the most problematic site"]},
    {"input": "What is the most problematic node", "output": ["find the most problematic node"]},
    {"input": "What are the recent correlated alarms", "output": ["find the recent correlated alarms"]},
    {"input": "What are the top affected nodes in site abc", "output": ["find the top affected nodes in a specific site"]},
    {"input": "What are the top reported alarms in node xyz", "output": ["find the top reported alarms in a specific node"]},
    {"input": "What are the sites having devices that have the risk of capacity overruns", "output": ["find the sites with devices having risk of capacity overruns"]},
    {"input": """Monitor bandwidth utilization metric in devices of type C, 
    verify the configuration parameters of misbehaving devices 
    and update the configurations once found""", "output": ["monitor specific metric in devices of a specific type", 
    "verify the configuration parameters of misbehaving devices", "update the configurations once found"]},
    {"input": "Provide the details of alarms reported in site A over last 2 hours?", "output": ["find the alarms reported in site A over a specific time period"]},
    {"input": "Monitor the network using RSRP and SINR metrics and ensure network is stable", "output": ["Monitor the network using specific metrics", 
     "ensure network is stable"]},
    {"input": "Raise a high priority ticket when alarm X and alarm Y are reported within a span of 5 minutes", 
     "output": ["Raise a high priority ticket when specific are reported within a specific time period"]}
]

In [4]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

In [5]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [6]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an expert network engineer who is provided with a user intent of the tasks to do. Your responsibility is to
        analyse the intent and break it down into individual task descriptions that the team can work on. 
        'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'
        """),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [7]:
prompt = prompt_template.format(input="""Monitor bandwidth utilization metric in devices on site A and site B, 
    verify the configuration parameters of misbehaving devices 
    and update the configurations if found""")

In [8]:
model = ChatOpenAI()
response=model.invoke(prompt)

In [9]:
response.content

"AI: ['monitor specific metric in devices on specific sites', 'verify the configuration parameters of misbehaving devices', 'update the configurations if found']"

In [10]:
embeddings = OpenAIEmbeddings()
docs = [
    "Find the most problematic site?",
    "Find the problematic node?",
    "Find the top affected nodes in a specific site?",
    "Find the recently correlated alarms?",
    "Find the top reported alarms in a specific node?",
    "Find if any anomalies have been reported on a specific node?",
    "Find the top affected sites?",
    "How to resolve the a specific alarm",
    "Why does a specific alarm occur?",
    "Monitor the network using specific metrics",
    "Ensure network is stable",
    "Raise a high priority ticket when specific alarms are reported within a specific time period",
    "Provide the details of alarms reported in a specific site over a specific time period",
    "Find the list of devices with license expiry due in a specific time period",
    "Monitor a specific metric in devices of specific type",
    "Verify the configuration parameters of misbaving devices",
    "Update the device configurations",
    "What are the sites having devices that have the risk of capacity overruns"
]

In [11]:
vec_store = Chroma.from_texts(texts = docs, embedding = embeddings)

In [12]:
vec_store.similarity_search('Why does link down occur', 3)

[Document(page_content='Why does a specific alarm occur?'),
 Document(page_content='Ensure network is stable'),
 Document(page_content='Find the problematic node?')]